In [1]:
import polars as pl
import os
import json

Reuse the processed dataset since the actors contained will already have been filtered.


In [2]:
data_dir = "/Users/maxwoolf/Downloads"

df = pl.read_parquet(
    os.path.join(data_dir, "test_movie_json_input.parquet"),
)

df

tconst,startYear,numVotes,averageRating,json
str,i64,i64,f64,str
"""tt9916362""",2020,6039,6.4,"""{ ""title"": ""Coven"", ""genre…"
"""tt9916270""",2020,1507,5.8,"""{ ""title"": ""Il talento del c…"
"""tt9916190""",2020,263,3.6,"""{ ""title"": ""Safeguard"", ""g…"
"""tt9916160""",2019,52,6.2,"""{ ""title"": ""Drømmeland"", ""…"
"""tt9915790""",2019,45,7.0,"""{ ""title"": ""Bobbyr Bondhura""…"
…,…,…,…,…
"""tt0000630""",1908,33,3.2,"""{ ""title"": ""Hamlet"", ""genr…"
"""tt0000591""",1907,31,5.6,"""{ ""title"": ""The Prodigal Son…"
"""tt0000574""",1906,985,6.0,"""{ ""title"": ""The Story of the…"


...but will need to process the JSON again. https://stackoverflow.com/a/73129057

The JSON is extracted as a `str` so must be converted to a `List[str]`. Which can be hacked with `json_decode()`


In [3]:
df_actors_test = (
    df.head(5)
    .with_columns(
        actors=pl.col("json").str.json_path_match(r"$.actors").str.json_decode()
    )
    .select(pl.col("actors"))
)

df_actors_test

actors
list[str]
"[""Amaia Aberasturi"", ""Alex Brendemühl"", … ""Elena Uriz""]"
"[""Sergio Castellitto"", ""Lorenzo Richelmy"", … ""Bianca Friscelli""]"
"[""Patrick Gallagher"", ""Akie Kotabe"", … ""Yuriri Naka""]"
[]
"[""Kaushik Sen"", ""Sreelekha Mitra"", … ""Bhaswar Chatterjee""]"


Run on the full set, and simultaneously aggregate metrics.


In [4]:
df_actors = (
    df.with_columns(
        actors=pl.col("json").str.json_path_match(r"$.actors").str.json_decode()
    )
    .explode("actors")
    .filter(pl.col("actors").is_not_null())
    .group_by("actors")
    .agg(
        movie_count=pl.col("actors").len(),
        avg_movie_rating=pl.col("averageRating").mean(),
    )
    .sort("movie_count", descending=True)
)

df_actors

actors,movie_count,avg_movie_rating
str,u32,f64
"""Brahmanandam""",572,5.992657
"""Eric Roberts""",410,4.65122
"""Jagathy Sreekumar""",367,5.819074
"""Shakti Kapoor""",338,5.171302
"""Anupam Kher""",331,5.532024
…,…,…
"""Thodoros Papadopoulos""",1,6.1
"""Anna Sniadówka""",1,5.6
"""Tate Ryder""",1,5.2


In [5]:
df_actors.filter(pl.col("actors") == "Keanu Reeves")

actors,movie_count,avg_movie_rating
str,u32,f64
"""Keanu Reeves""",69,6.266667


Per Reeves' IMDb, after filtering for Movies only, he has acted (at time of writing) in [85 movies](https://www.imdb.com/name/nm0000206/?ref_=nv_sr_srsg_0_tt_7_nm_1_in_0_q_keanu%2520reeves) w/ a rating, so this is close but may not be comprehensive / some movies may be filtered out due to lack of votes.


## Write to File for Data Viz


In [6]:
df_actors.write_parquet(os.path.join(data_dir, "actor_agg.parquet"))